# Get Domain

**Author:** Andrew Loeppky (Lots of code stolen from Jamie Byer)

**Project:** Land-surface-atmosphere coupling - CMIP6 intercomparison 

This code grabs a climate model from the cloud, screens it for required variable fields, then selects a user specified domain and saves it to disk as a netcdf4 file.

## Part I: Get a CMIP 6 Dataset and Select Domain

In [1]:
import xarray as xr
import pooch
import pandas as pd
import fsspec
from pathlib import Path
import time
import numpy as np
import json
import cftime
import matplotlib.pyplot as plt
import netCDF4 as nc


# Handy metpy tutorial working with xarray:
# https://unidata.github.io/MetPy/latest/tutorials/xarray_tutorial.html#sphx-glr-tutorials-xarray-tutorial-py
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.units import units
from metpy.plots import SkewT

In [89]:
# Attributes of the model we want to analyze (put in csv later)
source_id = 'CESM2-SE' 
#source_id = 'GFDL-ESM4' # working fig 11
#source_id = "CanESM5" 
#source_id = 'HadGEM3-GC31-MM'

#source_id = 'E3SM-1-0'
#source_id = 'INM-CM5-0'
#source_id = 'NorESM2-LM'
#source_id = 'GFDL-ESM4'
#source_id = 'MPI-ESM1-2-HR'

experiment_id = 'piControl'
#experiment_id = 'historical'
#table_id = 'Amon'
#table_id = '6hrLev'
table_id = '3hr'

# Domain we wish to study

# test domain #
##################################################################
lats = (15, 20) # lat min, lat max
lons = (25, 29) # lon min, lon max
years = (100, 105) # start year, end year (note, no leap days)
#ceil = 500 # top of domain, hPa
##################################################################

# Thompson, MB
lats = (54, 56) # lat min, lat max
lons = (261, 263) # lon min, lon max
years = (100, 300) # start year, end year (note, no leap days)
#ceil = 500 # top of domain, hPa

save_data = False # save as netcdf for further processing?

In [90]:
required_fields = ['tas', 'mrsos', 'huss', 'ps'] 

In [91]:
# get esm datastore
odie = pooch.create(
    path="./.cache",
    base_url="https://storage.googleapis.com/cmip6/",
    registry={
        "pangeo-cmip6.csv": None
    },
)
file_path = odie.fetch("pangeo-cmip6.csv")
df_in = pd.read_csv(file_path)

In [52]:
df_in.experiment_id.unique()

array(['highresSST-present', 'piControl', 'control-1950', 'hist-1950',
       'historical', 'amip', 'abrupt-4xCO2', 'abrupt-2xCO2',
       'abrupt-0p5xCO2', '1pctCO2', 'ssp585', 'esm-piControl', 'esm-hist',
       'hist-piAer', 'histSST-1950HC', 'ssp245', 'hist-1950HC', 'histSST',
       'piClim-2xVOC', 'piClim-2xNOx', 'piClim-2xdust', 'piClim-2xss',
       'piClim-histall', 'hist-piNTCF', 'histSST-piNTCF',
       'aqua-control-lwoff', 'piClim-lu', 'histSST-piO3', 'piClim-CH4',
       'piClim-NTCF', 'piClim-NOx', 'piClim-O3', 'piClim-HC',
       'faf-heat-NA0pct', 'ssp370SST-lowCH4', 'piClim-VOC',
       'ssp370-lowNTCF', 'piClim-control', 'piClim-aer', 'hist-aer',
       'faf-heat', 'faf-heat-NA50pct', 'ssp370SST-lowNTCF',
       'ssp370SST-ssp126Lu', 'ssp370SST', 'ssp370pdSST', 'histSST-piAer',
       'piClim-ghg', 'piClim-anthro', 'faf-all', 'hist-nat', 'hist-GHG',
       'ssp119', 'piClim-histnat', 'piClim-4xCO2', 'ssp370',
       'piClim-histghg', 'highresSST-future', 'esm-ssp585-

In [92]:
df_in[df_in.experiment_id == 'piControl'].table_id.unique()

array(['Amon', 'Lmon', 'fx', 'day', 'AERmon', '3hr', 'AERday', 'SImon',
       'Eday', 'EdayZ', 'Omon', 'CFday', 'CFmon', 'Emon', 'Oyr', 'SIday',
       'EmonZ', 'LImon', 'Oday', 'Ofx', 'AERmonZ', '6hrPlevPt', 'Efx',
       'Aclim', 'Odec', 'Oclim', 'SIclim', 'Eyr', 'CF3hr', 'IfxGre'],
      dtype=object)

In [93]:
available_fields = df_in[df_in.experiment_id == 'piControl'][df_in.table_id == '3hr'].variable_id.unique()
available_fields

/tmp/ipykernel_7788/1805301981.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  available_fields = df_in[df_in.experiment_id == 'piControl'][df_in.table_id == '3hr'].variable_id.unique()


array(['tas', 'mrsos', 'mrro', 'tslsi', 'huss', 'rsdsdiff', 'rsdscs',
       'rsds', 'rlus', 'rldscs', 'rlds', 'ps', 'prsn', 'prc', 'rsuscs',
       'rsus', 'uas', 'pr', 'vas', 'hfls', 'clt', 'hfss', 'tos'],
      dtype=object)

In [94]:
# check that our run has all required fields, list problem variables
fields_of_interest = []
missing_fields = []
for rq in required_fields:
    if rq not in available_fields:
        missing_fields.append(rq)
    else:
        fields_of_interest.append(rq)


print(f"Model: {source_id}\n"+"="*30)
print("Contains required fields:")
[print("   ", field) for field in required_fields if field in fields_of_interest]

if fields_of_interest == required_fields:
    model_passes = True
    print("All required fields present")
else: 
    model_passes = False
    print("Missing required fields:")
    [print("   ", field) for field in required_fields if field not in fields_of_interest]
        

Model: CESM2-SE
Contains required fields:
    tas
    mrsos
    huss
    ps
All required fields present


In [95]:
print("*" * 50)
print(f"""Fetching domain:
          {source_id = }
          {experiment_id = }
          {table_id = }
          {lats = }
          {lons = }
          {years = }
          dataset name: my_ds (xarray Dataset)""")
print("\n"+"*" * 50, "\n")

**************************************************
Fetching domain:
          source_id = 'CESM2-SE'
          experiment_id = 'piControl'
          table_id = '3hr'
          lats = (54, 56)
          lons = (261, 263)
          years = (100, 300)
          dataset name: my_ds (xarray Dataset)

************************************************** 



In [9]:
def fetch_var_exact(the_dict,df_og):
    the_keys = list(the_dict.keys())
    #print(the_keys)
    key0 = the_keys[0]
    #print(key0)
    #print(the_dict[key0])
    hit0 = df_og[key0] == the_dict[key0]
    if len(the_keys) > 1:
        hitnew = hit0
        for key in the_keys[1:]:
            hit = df_og[key] == the_dict[key]
            hitnew = np.logical_and(hitnew,hit)
            #print("total hits: ",np.sum(hitnew))
    else:
        hitnew = hit0
    df_result = df_og[hitnew]
    return df_result

In [10]:
def get_field(variable_id, 
              df,
              source_id=source_id,
              experiment_id=experiment_id,
              table_id=table_id):
    """
    extracts a single variable field from the model
    """

    var_dict = dict(source_id = source_id, variable_id = variable_id,
                    experiment_id = experiment_id, table_id = table_id)
    
    local_var = fetch_var_exact(var_dict, df)
    zstore_url = local_var['zstore'].array[0]
    the_mapper=fsspec.get_mapper(zstore_url)
    local_var = xr.open_zarr(the_mapper, consolidated=True)
    return local_var

In [11]:
def trim_field(df, lat, lon, years):
    """
    cuts out a specified domain from an xarrray field
    
    lat = (minlat, maxlat)
    lon = (minlon, maxlon)
    """
    new_field = df.sel(lat=slice(lat[0],lat[1]), lon=slice(lon[0],lon[1]))
    new_field = new_field.isel(time=(new_field.time.dt.year > years[0]))
    new_field = new_field.isel(time=(new_field.time.dt.year < years[1]))
    return new_field

In [12]:
# grab all fields of interest and combine
my_fields = [get_field(field, df_in) for field in fields_of_interest]
small_fields = [trim_field(field, lats, lons, years) for field in my_fields]
my_ds = xr.combine_by_coords(small_fields, compat="broadcast_equals", combine_attrs="drop_conflicts")
print("Successfully acquired domain")

Successfully acquired domain


In [13]:
from cftime import date2num
#date2num(my_ds.time, "minutes since 0000-01-01 00:00:00", calendar="noleap", has_year_zero=True)
my_ds

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*

In [14]:
# save as netcdf as per these recommendations:
# https://xarray.pydata.org/en/stable/user-guide/dask.html#chunking-and-performance
# netcdf cant handle cftime, so convert to ordinal, then back once the file is reopened
my_ds["time"] = date2num(my_ds.time, "minutes since 0000-01-01 00:00:00", calendar="noleap", has_year_zero=True)
my_ds = my_ds.drop("time_bnds")

AttributeError: 'Dataset' object has no attribute 'time'

In [ ]:
if save_data:
    my_ds.to_netcdf(f"./data/{source_id}-{experiment_id}.nc", engine="netcdf4")